In [37]:
import pandas as pd
from sodapy import Socrata
from dotenv import load_dotenv
import os
from datetime import datetime
import psycopg2 as pg2

In [38]:
load_dotenv()
myapptoken = os.getenv('MYAPPTOKEN')
postgres_user = os.getenv('POSTGRES_USER')
postgres_password = os.getenv('POSTGRES_PASSWORD')
client = Socrata("data.ny.gov",
                 myapptoken
                 )


In [39]:
results = client.get("qzve-kjga", where="plaza_id = 30", limit=100000)
results_df = pd.DataFrame.from_records(results)

In [47]:
results_df = results_df.loc[0]
results_df

plaza_id                                 30
date                2024-10-19T00:00:00.000
hour                                      0
direction                                 I
vehicles_e_zpass                       1604
vehicles_vtoll                          285
Name: 0, dtype: object

In [65]:
connection = pg2.connect(dbname='MTA Open Data', user=postgres_user, password=postgres_password)

In [66]:
cur = connection.cursor()

create_table_query = """ 
CREATE TABLE IF NOT EXISTS traffic 
    (
        plaza_id SMALLINT,
        date TIMESTAMP,
        hour SMALLINT,
        direction text,
        vehicles_ezpass integer,
        vehicles_vtoll integer
    )

"""

insert_query = """
    INSERT INTO traffic (plaza_id, date, hour, direction, vehicles_ezpass, vehicles_vtoll)
    VALUES (%s, %s, %s, %s, %s, %s)

"""

select_query = "SELECT * FROM traffic"

cur.execute(create_table_query)
cur.execute(insert_query, (results_df['plaza_id'], results_df['date'], results_df['hour'], results_df['direction'], results_df['vehicles_e_zpass'], results_df['vehicles_vtoll']))
cur.execute(select_query)
print(cur.fetchone())
connection.commit()

cur.close()
connection.close()
    


(30, datetime.datetime(2024, 10, 19, 0, 0), 0, 'I', 1604, 285)
